In [10]:
import PIL  #(Pillow)
from PIL import Image
import numpy as np 
import pandas as pd
import os
import cv2
import imageio
import dlib
import seaborn as sns
import json
import random
import torchvision
import torchvision.transforms.functional as FT
import torch
from torch import nn
from torch.utils.data import Dataset
import torch.backends.cudnn as cudnn
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import skimage 
from skimage.transform import resize
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize,rescale
from urllib.request import urlopen

In [11]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from math import exp, log10, sqrt

class PSNR:
    """Peak Signal to Noise Ratio
    img1 and img2 have range [0, 255]"""

    def __init__(self):
        self.name = "PSNR"

    @staticmethod
    def __call__(img1, img2):
        mse = torch.mean((img1 - img2) ** 2)
        return 20 * torch.log10(1.0 / torch.sqrt(mse))

def psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    pixel_max = 1.0
    return 20 * log10(pixel_max / sqrt(mse))



def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size // 2) ** 2 / float(2 * sigma ** 2)) for x in range(window_size)])
    return gauss / gauss.sum()


def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window


def _ssim(img1, img2, window, window_size, channel, size_average=True):
    mu1 = F.conv2d(img1, window, padding=window_size // 2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size // 2, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=window_size // 2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=window_size // 2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=window_size // 2, groups=channel) - mu1_mu2

    C1 = 0.01 ** 2
    C2 = 0.03 ** 2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)


class SSIM(torch.nn.Module):
    def __init__(self, window_size=11, size_average=True):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.channel = 1
        self.window = create_window(window_size, self.channel)
     
    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()

        if channel == self.channel and self.window.data.type() == img1.data.type():
            window = self.window
        else:
            window = create_window(self.window_size, channel)

            if img1.is_cuda:
                window = window.cuda(img1.get_device())
            window = window.type_as(img1)

            self.window = window
            self.channel = channel

        return _ssim(img1, img2, window, self.window_size, channel, self.size_average)


def ssim(img1, img2, window_size=11, size_average=True):
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)

    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)

    return _ssim(img1, img2, window, window_size, channel, size_average)


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [12]:
from PIL import Image
import os
import json
import random
import torchvision.transforms.functional as FT
import torch
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Some constants
rgb_weights = torch.FloatTensor([65.481, 128.553, 24.966]).to(device)
rgb_weights = torch.FloatTensor([0.299, 0.587, 0.114]).to(device)


def convert_image(img, source, target):
    """
    Convert an image from a source format to a target format.
    :param img: image
    :param source: source format, one of 'pil' (PIL image), '[0, 1]' or '[-1, 1]' (pixel value ranges)
    :param target: target format, one of 'pil' (PIL image), '[0, 255]', '[0, 1]', '[-1, 1]' (pixel value ranges),
                   'y-channel' (luminance channel Y in the YCbCr color format, used to calculate PSNR and SSIM)
    :return: converted image
    """
    assert source in {'pil', '[0, 1]', '[-1, 1]'}, "Cannot convert from source format %s!" % source
    assert target in {'pil', '[0, 255]', '[0, 1]', '[-1, 1]',
                      'y-channel'}, "Cannot convert to target format %s!" % target

    # Convert from source to [0, 1]
    if source == 'pil':
        img = FT.to_tensor(img)

    elif source == '[0, 1]':
        pass  # already in [0, 1]

    elif source == '[-1, 1]':
        img = (img + 1.) / 2.

    # Convert from [0, 1] to target
    if target == 'pil':
        img = FT.to_pil_image(img)

    elif target == '[0, 255]':
        img = 255. * img

    elif target == '[0, 1]':
        pass  # already in [0, 1]

    elif target == '[-1, 1]':
        img = 2. * img - 1.

    elif target == 'y-channel':
        img = torch.matmul(img.permute(0, 2, 3, 1)[:, :, :, :], rgb_weights)

    return img

class ImageTransforms(object):
    """
    Image transformation pipeline.
    """

    def __init__(self, split, crop_size, scaling_factor, lr_img_type, hr_img_type):
        """
        :param split: one of 'train' or 'test'
        :param crop_size: crop size of HR images
        :param scaling_factor: LR images will be downsampled from the HR images by this factor
        :param lr_img_type: the target format for the LR image; see convert_image() above for available formats
        :param hr_img_type: the target format for the HR image; see convert_image() above for available formats
        """
        self.split = split.lower()
        self.crop_size = crop_size
        self.scaling_factor = scaling_factor
        self.lr_img_type = lr_img_type
        self.hr_img_type = hr_img_type

        assert self.split in {'train', 'valid', 'test'}

    def __call__(self, img):
        """
        :param img: a PIL source image from which the HR image will be cropped, and then downsampled to create the LR image
        :return: LR and HR images in the specified format
        """

        # Crop
        if self.split == 'train':
            # Take a random fixed-size crop of the image, which will serve as the high-resolution (HR) image
            left = random.randint(1, img.width - self.crop_size)
            top = random.randint(1, img.height - self.crop_size)
            right = left + self.crop_size
            bottom = top + self.crop_size
            hr_img = img.crop((left, top, right, bottom))
        elif self.split == 'valid':
            # Take a random fixed-size crop of the image, which will serve as the high-resolution (HR) image
            left = random.randint(1, img.width - self.crop_size)
            top = random.randint(1, img.height - self.crop_size)
            right = left + self.crop_size
            bottom = top + self.crop_size
            hr_img = img.crop((left, top, right, bottom))
        else:
            # Take the largest possible center-crop of it such that its dimensions are perfectly divisible by the scaling factor
            x_remainder = img.width % self.scaling_factor
            y_remainder = img.height % self.scaling_factor
            left = x_remainder // 2
            top = y_remainder // 2
            right = left + (img.width - x_remainder)
            bottom = top + (img.height - y_remainder)
            hr_img = img.crop((left, top, right, bottom))

        lr_img = hr_img.resize((int(hr_img.width / self.scaling_factor), int(hr_img.height / self.scaling_factor)),
                               Image.BICUBIC)

        assert hr_img.width == lr_img.width * self.scaling_factor and hr_img.height == lr_img.height * self.scaling_factor

        lr_img = convert_image(lr_img, source='pil', target=self.lr_img_type)
        hr_img = convert_image(hr_img, source='pil', target=self.hr_img_type)

        return lr_img, hr_img

import numpy as np 
import pandas as pd

In [13]:
#1 first
class ConvolutionalBlock(nn.Module):
    """
    A convolutional block, comprising convolutional, Batch Normalization, activation layers.
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, batch_norm=False, activation=None):
        """
        :param in_channels: number of input channels
        :param out_channels: number of output channels
        :param kernel_size: kernel size
        :param stride: stride
        :param batch_norm: include a Batch Normalization (BN) layer?
        :param activation: Type of activation; None if none
        """
        super(ConvolutionalBlock, self).__init__()

        if activation is not None:
            activation = activation.lower()
            assert activation in {'prelu', 'leakyrelu', 'tanh', 'sigmoid'}

        # A container that will hold the layers in this convolutional block
        layers = list()

        # A convolutional layer
        layers.append(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                                stride=stride,padding=kernel_size // 2))

        # A batch normalization (BN) layer, if wanted
        if batch_norm is True:
            layers.append(nn.BatchNorm2d(num_features=out_channels))

        # An activation layer, if wanted
        if activation == 'prelu':
            layers.append(nn.PReLU())
        elif activation == 'leakyrelu':
            layers.append(nn.LeakyReLU(0.2))
        elif activation == 'tanh':
            layers.append(nn.Tanh())

        # Put together the convolutional block as a sequence of the layers in this container
        self.conv_block = nn.Sequential(*layers)

    def forward(self, input):
        """
        Forward propagation.
        :param input: input images, a tensor of size (N, in_channels, w, h)
        :return: output images, a tensor of size (N, out_channels, w, h)
        """
        output = self.conv_block(input)  # (N, out_channels, w, h)

        return output

#2 second

class SubPixelConvolutionalBlock(nn.Module):
    """
    A subpixel convolutional block, comprising convolutional, pixel-shuffle, and PReLU activation layers.
    """

    def __init__(self, kernel_size=3, n_channels=64, scaling_factor=2):
        """
        :param kernel_size: kernel size of the convolution
        :param n_channels: number of input and output channels
        :param scaling_factor: factor to scale input images by (along both dimensions)
        """
        super(SubPixelConvolutionalBlock, self).__init__()

        # A convolutional layer that increases the number of channels by scaling factor^2, followed by pixel shuffle and PReLU
        self.conv = nn.Conv2d(in_channels=n_channels, out_channels=n_channels * (scaling_factor ** 2),
                              kernel_size=kernel_size, padding=kernel_size // 2)
        # These additional channels are shuffled to form additional pixels, upscaling each dimension by the scaling factor
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor=scaling_factor)
        self.prelu = nn.PReLU()

    def forward(self, input):
        """
        Forward propagation.
        :param input: input images, a tensor of size (N, n_channels, w, h)
        :return: scaled output images, a tensor of size (N, n_channels, w * scaling factor, h * scaling factor)
        """
        output = self.conv(input)  # (N, n_channels * scaling factor^2, w, h)
        output = self.pixel_shuffle(output)  # (N, n_channels, w * scaling factor, h * scaling factor)
        output = self.prelu(output)  # (N, n_channels, w * scaling factor, h * scaling factor)

        return output

#3 third
class ResidualBlock(nn.Module):
    """
    A residual block, comprising two convolutional blocks with a residual connection across them.
    """

    def __init__(self, kernel_size=3, n_channels=64):
        """
        :param kernel_size: kernel size
        :param n_channels: number of input and output channels (same because the input must be added to the output)
        """
        super(ResidualBlock, self).__init__()

        # The first convolutional block
        self.conv_block1 = ConvolutionalBlock(in_channels=n_channels, out_channels=n_channels, kernel_size=kernel_size,
                                              batch_norm=True, activation='PReLu')

        # The second convolutional block
        self.conv_block2 = ConvolutionalBlock(in_channels=n_channels, out_channels=n_channels, kernel_size=kernel_size,
                                              batch_norm=True, activation=None)

    def forward(self, input):
        """
        Forward propagation.
        :param input: input images, a tensor of size (N, n_channels, w, h)
        :return: output images, a tensor of size (N, n_channels, w, h)
        """
        residual = input  # (N, n_channels, w, h)
        output = self.conv_block1(input)  # (N, n_channels, w, h)
        output = self.conv_block2(output)  # (N, n_channels, w, h)
        output = output + residual  # (N, n_channels, w, h)

        return output


#4 fourth
class SRResNet(nn.Module):
    """
    The SRResNet, as defined in the  paper.
    """

    def __init__(self, large_kernel_size=9, small_kernel_size=3, n_channels=64, n_blocks=16, scaling_factor=4):
        """
        :param large_kernel_size: kernel size of the first and last convolutions which transform the inputs and outputs
        :param small_kernel_size: kernel size of all convolutions in-between, i.e. those in the residual and subpixel convolutional blocks
        :param n_channels: number of channels in-between, i.e. the input and output channels for the residual and subpixel convolutional blocks
        :param n_blocks: number of residual blocks
        :param scaling_factor: factor to scale input images by (along both dimensions) in the subpixel convolutional block
        """
        super(SRResNet, self).__init__()

        # Scaling factor must be 2, 4, or 8
        scaling_factor = int(scaling_factor)
        assert scaling_factor in {2, 4, 8}, "The scaling factor must be 2, 4, or 8!"

        # The first convolutional block
        self.conv_block1 = ConvolutionalBlock(in_channels=3, out_channels=n_channels, kernel_size=large_kernel_size,
                                              batch_norm=False, activation='PReLu')

        # A sequence of n_blocks residual blocks, each containing a skip-connection across the block
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(kernel_size=small_kernel_size, n_channels=n_channels) for i in range(n_blocks)])

        # Another convolutional block
        self.conv_block2 = ConvolutionalBlock(in_channels=n_channels, out_channels=n_channels,
                                              kernel_size=small_kernel_size,
                                              batch_norm=True, activation=None)

        # Upscaling is done by sub-pixel convolution, with each such block upscaling by a factor of 2
        n_subpixel_convolution_blocks = int(math.log2(scaling_factor))
        self.subpixel_convolutional_blocks = nn.Sequential(
            *[SubPixelConvolutionalBlock(kernel_size=small_kernel_size, n_channels=n_channels, scaling_factor=2) for i
              in range(n_subpixel_convolution_blocks)])

        # The last convolutional block
        self.conv_block3 = ConvolutionalBlock(
            in_channels=n_channels, 
            out_channels=3, 
            kernel_size=large_kernel_size,
            batch_norm=False, 
            activation='tanh')

    def forward(self, lr_imgs):
        """
        Forward prop.
        :param lr_imgs: low-resolution input images, a tensor of size (N, 3, w, h)
        :return: super-resolution output images, a tensor of size (N, 3, w * scaling factor, h * scaling factor)
        """
        output = self.conv_block1(lr_imgs)  # (N, 3, w, h)
        residual = output  # (N, n_channels, w, h)
        output = self.residual_blocks(output)  # (N, n_channels, w, h)
        output = self.conv_block2(output)  # (N, n_channels, w, h)
        output = output + residual  # (N, n_channels, w, h)
        output = self.subpixel_convolutional_blocks(output)  # (N, n_channels, w * scaling factor, h * scaling factor)
        sr_imgs = self.conv_block3(output)  # (N, 3, w * scaling factor, h * scaling factor)

        return sr_imgs

#5  fifth
class Generator(nn.Module):  #(SRGAN)
    """
    The generator in the SRGAN, as defined in the paper. Architecture identical to the SRResNet.
    """

    def __init__(self, large_kernel_size=9, small_kernel_size=3, n_channels=64, n_blocks=16, scaling_factor=4):
        """
        :param large_kernel_size: kernel size of the first and last convolutions which transform the inputs and outputs
        :param small_kernel_size: kernel size of all convolutions in-between, i.e. those in the residual and subpixel convolutional blocks
        :param n_channels: number of channels in-between, i.e. the input and output channels for the residual and subpixel convolutional blocks
        :param n_blocks: number of residual blocks
        :param scaling_factor: factor to scale input images by (along both dimensions) in the subpixel convolutional block
        """
        super(Generator, self).__init__()

        # The generator is simply an SRResNet, as above
        self.net = SRResNet(large_kernel_size=large_kernel_size, small_kernel_size=small_kernel_size,
                            n_channels=n_channels, n_blocks=n_blocks, scaling_factor=scaling_factor)

    def initialize_with_srresnet(self, srresnet_checkpoint):
        """
        Initialize with weights from a trained SRResNet.
        :param srresnet_checkpoint: checkpoint filepath
        """
        srresnet = torch.load(srresnet_checkpoint)['model']
        self.net.load_state_dict(srresnet.state_dict())

        print("\nLoaded weights from pre-trained SRResNet.\n")

    def forward(self, lr_imgs):
        """
        Forward prop.
        :param lr_imgs: low-resolution input images, a tensor of size (N, 3, w, h)
        :return: super-resolution output images, a tensor of size (N, 3, w * scaling factor, h * scaling factor)
        """
        sr_imgs = self.net(lr_imgs)  # (N, n_channels, w * scaling factor, h * scaling factor)

        return sr_imgs

#6 sixth
class Discriminator(nn.Module):
    """
    The discriminator in the SRGAN, as defined in the paper.
    """

    def __init__(self, kernel_size=3, n_channels=64, n_blocks=8, fc_size=1024):
        """
        :param kernel_size: kernel size in all convolutional blocks
        :param n_channels: number of output channels in the first convolutional block, after which it is doubled in every 2nd block thereafter
        :param n_blocks: number of convolutional blocks
        :param fc_size: size of the first fully connected layer
        """
        # A series of convolutional blocks
        # The first, third, fifth (and so on) convolutional blocks increase the number of channels but retain image size
        # The second, fourth, sixth (and so on) convolutional blocks retain the same number of channels but halve image size
        # The first convolutional block is unique because it does not employ batch normalization.
        
        super(Discriminator, self).__init__()

        in_channels = 3
        conv_blocks = list()
        for i in range(n_blocks):
            out_channels = (n_channels if i is 0 else in_channels * 2) if i % 2 is 0 else in_channels
            conv_blocks.append(
                ConvolutionalBlock(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                                   stride=1 if i % 2 is 0 else 2, batch_norm=i is not 0, activation='LeakyReLu'))
            in_channels = out_channels
        self.conv_blocks = nn.Sequential(*conv_blocks)

        # An adaptive pool layer that resizes it to a standard size
        # For the default input size of 96 and 8 convolutional blocks, this will have no effect
        
        self.adaptive_pool = nn.AdaptiveAvgPool2d((6, 6))

        self.fc1 = nn.Linear(out_channels * 6 * 6, fc_size)

        self.leaky_relu = nn.LeakyReLU(0.2)

        self.fc2 = nn.Linear(1024, 1)

        # Don't need a sigmoid layer because the sigmoid operation is performed by PyTorch's nn.BCEWithLogitsLoss()

    def forward(self, imgs):
        """
        Forward propagation.
        :param imgs: high-resolution or super-resolution images which must be classified as such, a tensor of size (N, 3, w * scaling factor, h * scaling factor)
        :return: a score (logit) for whether it is a high-resolution image, a tensor of size (N)
        """
        batch_size = imgs.size(0)
        output = self.conv_blocks(imgs)
        output = self.adaptive_pool(output)
        output = self.fc1(output.view(batch_size, -1))
        output = self.leaky_relu(output)
        logit = self.fc2(output)

        return logit
    

class TruncatedVGG19(nn.Module):
    """
    A truncated VGG19 network, such that its output is the 'feature map obtained by the j-th convolution (after activation)
    before the i-th maxpooling layer within the VGG19 network', as defined in the paper.
    Used to calculate the (MSE) loss in this VGG feature-space, i.e. the VGG loss.
    """

    def __init__(self, i, j):
        """
        :param i: the index i in the definition above
        :param j: the index j in the definition above
        """
        super(TruncatedVGG19, self).__init__()

        # Load the pre-trained VGG19 available in torchvision
        vgg19 = torchvision.models.vgg19(pretrained=True)

        maxpool_counter = 0
        conv_counter = 0
        truncate_at = 0
        # Iterate through the convolutional section ("features") of the VGG19
        for layer in vgg19.features.children():
            truncate_at += 1

            # Count the number of maxpool layers and the convolutional layers after each maxpool
            if isinstance(layer, nn.Conv2d):
                conv_counter += 1
            if isinstance(layer, nn.MaxPool2d):
                maxpool_counter += 1
                conv_counter = 0

            if maxpool_counter == i - 1 and conv_counter == j:
                break

        assert maxpool_counter == i - 1 and conv_counter == j, "One or both of i=%d and j=%d are not valid choices for the VGG19!" % (
            i, j)

        self.truncated_vgg19 = nn.Sequential(*list(vgg19.features.children())[:truncate_at + 1])

    def forward(self, input):
        """
        Forward propagation
        :param input: high-resolution or super-resolution images, a tensor of size (N, 3, w * scaling factor, h * scaling factor)
        :return: the specified VGG19 feature map, a tensor of size (N, feature_map_channels, feature_map_w, feature_map_h)
        """
        output = self.truncated_vgg19(input)  # (N, feature_map_channels, feature_map_w, feature_map_h)

        return output

In [14]:
path_to_images = '../input/div2k-dataset/DIV2K_valid_HR/DIV2K_valid_HR'
pathes_to_images = [os.path.join(path_to_images, file) for file in os.listdir(path_to_images)]

checkpoint = torch.load('../input/weightsresnet/checkpoint_srresnet.pth.tar')
resnet = checkpoint['model'].eval()

checkpoint = torch.load('../input/sr-gan/checkpoint_srgan.pth.tar')
resnet_gan = checkpoint['generator']

transform = ImageTransforms(split='valid',
                            crop_size=512,
                            scaling_factor=4,
                            lr_img_type='pil',
                            hr_img_type='pil')

In [15]:
avg_psnr_bilinear = AverageMeter()
avg_ssim_bilinear = AverageMeter()
avg_psnr_bicubic = AverageMeter()
avg_ssim_bicubic = AverageMeter()
avg_psnr_resnet = AverageMeter()
avg_ssim_resnet = AverageMeter()
avg_psnr_srgan = AverageMeter()
avg_ssim_srgan = AverageMeter()

toPIL = torchvision.transforms.ToPILImage()
ssim_metric = SSIM()

for file in pathes_to_images:
    img = Image.open(file, mode='r')
    img = img.convert('RGB')
    lr_img, hr_img = transform(img)
    
    img_bilinear = convert_image(lr_img.resize(hr_img.size, Image.BILINEAR), 'pil', '[-1, 1]')
    img_bicubic = convert_image(lr_img.resize(hr_img.size, Image.BICUBIC), 'pil', '[-1, 1]')
    img_bilinear = convert_image(torch.tensor(img_bilinear).unsqueeze(0).to(device), '[-1, 1]', 'y-channel')
    img_bicubic = convert_image(torch.tensor(img_bicubic).unsqueeze(0).to(device), '[-1, 1]', 'y-channel')
    
    lr_img = convert_image(lr_img, 'pil', '[-1, 1]')
    hr_img = convert_image(hr_img, 'pil', '[-1, 1]').unsqueeze(0).to(device)
    hr_img = convert_image(hr_img, '[-1, 1]', 'y-channel').squeeze()
    
    img_pred_resnet = resnet(torch.tensor(lr_img).unsqueeze(0).to(device))
    img_pred_srgan = resnet_gan(torch.tensor(lr_img).unsqueeze(0).to(device))
    img_pred_resnet = convert_image(img_pred_resnet, '[-1, 1]', 'y-channel').squeeze()#.detach().cpu().numpy()
    img_pred_srgan = convert_image(img_pred_srgan, '[-1, 1]', 'y-channel').squeeze()#.detach().cpu().numpy()
    
    avg_psnr_bilinear.update(psnr(hr_img, img_bilinear))
    avg_psnr_bicubic.update(psnr(hr_img, img_bicubic))
    avg_psnr_resnet.update(psnr(hr_img, img_pred_resnet))
    avg_psnr_srgan.update(psnr(hr_img, img_pred_srgan))
    
    avg_ssim_bilinear.update(ssim_metric(hr_img.unsqueeze(0).unsqueeze(0), img_bilinear.unsqueeze(0)).item())
    avg_ssim_bicubic.update(ssim_metric(hr_img.unsqueeze(0).unsqueeze(0), img_bicubic.unsqueeze(0)).item())
    avg_ssim_resnet.update(ssim_metric(hr_img.unsqueeze(0).unsqueeze(0), img_pred_resnet.unsqueeze(0)).item())
    avg_ssim_srgan.update(ssim_metric(hr_img.unsqueeze(0).unsqueeze(0), img_pred_srgan.unsqueeze(0)).item())

In [16]:
print('-' * 20 + 'PSNR' + '-' * 20)
print("bilinear: {:.3f}, bicubic: {:.3f}, resnet: {:.3f}, srgan: {:.3f}" \
      .format(avg_psnr_bilinear.avg, avg_psnr_bicubic.avg, avg_psnr_resnet.avg, avg_psnr_srgan.avg))

print('-' * 20 + 'SSIM' + '-' * 20)
print("bilinear: {:.3f}, bicubic: {:.3f}, resnet: {:.3f}, srgan: {:.3f}" \
      .format(avg_ssim_bilinear.avg, avg_ssim_bicubic.avg, avg_ssim_resnet.avg, avg_ssim_srgan.avg))

In [ ]:
img = imread('https://www.popsci.com/uploads/2019/01/07/GMY7X5OKPSBGSTSFEZXDEYW6JU-1024x1024.jpg').astype(float) / 255

bbox = [150, 150, 350, 350]

img_lr = resize(img,(int(img.shape[0]/4), int(img.shape[1]/4)))
plt.figure(figsize=(30,15))
plt.subplot(241)
plt.imshow(img_lr)
plt.title('Low resolution image')
plt.axis('off')
plt.gca().add_patch(patches.Rectangle(
    (int(bbox[1] / 4), int(bbox[0] / 4)), 
    int(bbox[3] / 4) - int(bbox[1] / 4), 
    int(bbox[2] / 4) - int(bbox[0] / 4),
    linewidth=3,
    edgecolor='r',
    facecolor='none'))

plt.subplot(245)
plt.imshow(img_lr[int(bbox[0] / 4):int(bbox[2] / 4), int(bbox[1] / 4):int(bbox[3] / 4)])
plt.axis('off')

plt.subplot(242)
plt.imshow(img)
plt.title('Original image')
plt.axis('off')
plt.gca().add_patch(patches.Rectangle(
    (int(bbox[1]), int(bbox[0])), 
    int(bbox[3]) - int(bbox[1]), 
    int(bbox[2]) - int(bbox[0]),
    linewidth=3,
    edgecolor='r',
    facecolor='none'))

plt.subplot(246)
plt.imshow(img[bbox[0]:bbox[2], bbox[1]:bbox[3]])
plt.axis('off')

img_resnet = resnet(torch.tensor(img_lr).swapaxes(1,2).swapaxes(0,1).unsqueeze(0).to(device).float()) \
.detach().cpu().squeeze().numpy().swapaxes(0,1).swapaxes(1,2)
img_gan = resnet_gan(torch.tensor(img_lr).swapaxes(1,2).swapaxes(0,1).unsqueeze(0).to(device).float()) \
.detach().cpu().squeeze().numpy().swapaxes(0,1).swapaxes(1,2)

plt.subplot(243)
plt.imshow(img_resnet)
plt.title('ResNet')
plt.axis('off')
plt.gca().add_patch(patches.Rectangle(
    (int(bbox[1]), int(bbox[0])), 
    int(bbox[3]) - int(bbox[1]), 
    int(bbox[2]) - int(bbox[0]),
    linewidth=3,
    edgecolor='r',
    facecolor='none'))

plt.subplot(247)
plt.imshow(img_resnet[bbox[0]:bbox[2], bbox[1]:bbox[3]])
plt.axis('off')

plt.subplot(244)
plt.imshow(img_gan)
plt.title('GAN Resnet')
plt.axis('off')
plt.gca().add_patch(patches.Rectangle(
    (int(bbox[1]), int(bbox[0])), 
    int(bbox[3]) - int(bbox[1]), 
    int(bbox[2]) - int(bbox[0]),
    linewidth=3,
    edgecolor='r',
    facecolor='none'))

plt.subplot(248)
plt.imshow(img_gan[bbox[0]:bbox[2], bbox[1]:bbox[3]])
plt.axis('off')

plt.savefig('earth.png', dpi=300, bbox_inches='tight')

In [ ]:
img = imread('https://www.sunhome.ru/i/wallpapers/85/tigr.xxl.jpg').astype(float) / 255

bbox = [150, 150, 350, 350]

img_lr = resize(img,(int(img.shape[0]/4), int(img.shape[1]/4)))
plt.figure(figsize=(30,15))
plt.subplot(241)
plt.imshow(img_lr)
plt.title('Low resolution image')
plt.axis('off')
plt.gca().add_patch(patches.Rectangle(
    (int(bbox[1] / 4), int(bbox[0] / 4)), 
    int(bbox[3] / 4) - int(bbox[1] / 4), 
    int(bbox[2] / 4) - int(bbox[0] / 4),
    linewidth=3,
    edgecolor='r',
    facecolor='none'))

plt.subplot(245)
plt.imshow(img_lr[int(bbox[0] / 4):int(bbox[2] / 4), int(bbox[1] / 4):int(bbox[3] / 4)])
plt.axis('off')

plt.subplot(242)
plt.imshow(img)
plt.title('Original image')
plt.axis('off')
plt.gca().add_patch(patches.Rectangle(
    (int(bbox[1]), int(bbox[0])), 
    int(bbox[3]) - int(bbox[1]), 
    int(bbox[2]) - int(bbox[0]),
    linewidth=3,
    edgecolor='r',
    facecolor='none'))

plt.subplot(246)
plt.imshow(img[bbox[0]:bbox[2], bbox[1]:bbox[3]])
plt.axis('off')

img_resnet = resnet(torch.tensor(img_lr).swapaxes(1,2).swapaxes(0,1).unsqueeze(0).to(device).float()) \
.detach().cpu().squeeze().numpy().swapaxes(0,1).swapaxes(1,2)
img_gan = resnet_gan(torch.tensor(img_lr).swapaxes(1,2).swapaxes(0,1).unsqueeze(0).to(device).float()) \
.detach().cpu().squeeze().numpy().swapaxes(0,1).swapaxes(1,2)

plt.subplot(243)
plt.imshow(img_resnet)
plt.title('ResNet')
plt.axis('off')
plt.gca().add_patch(patches.Rectangle(
    (int(bbox[1]), int(bbox[0])), 
    int(bbox[3]) - int(bbox[1]), 
    int(bbox[2]) - int(bbox[0]),
    linewidth=3,
    edgecolor='r',
    facecolor='none'))

plt.subplot(247)
plt.imshow(img_resnet[bbox[0]:bbox[2], bbox[1]:bbox[3]])
plt.axis('off')

plt.subplot(244)
plt.imshow(img_gan)
plt.title('GAN Resnet')
plt.axis('off')
plt.gca().add_patch(patches.Rectangle(
    (int(bbox[1]), int(bbox[0])), 
    int(bbox[3]) - int(bbox[1]), 
    int(bbox[2]) - int(bbox[0]),
    linewidth=3,
    edgecolor='r',
    facecolor='none'))

plt.subplot(248)
plt.imshow(img_gan[bbox[0]:bbox[2], bbox[1]:bbox[3]])
plt.axis('off')

plt.savefig('tiger.png', dpi=300, bbox_inches='tight')
